In [1]:
from transformers import BertModel,AutoTokenizer, FlaubertModel, BertForSequenceClassification , FlaubertForSequenceClassification
from transformers.modeling_utils import SequenceSummary
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.nn.utils.rnn import pack_padded_sequence
import sys
import re
from easy_nlp.models import ThreeTaskLearning
from easy_nlp.training import Train
from easy_nlp.preprocessing import TextPreprocessing
from easy_nlp.feature_extraction import MetaFeaturesExtraction
from easy_nlp.data_visualisation import WordCloudMaker
from transformers import  AutoModel


/home/aboumada/anaconda3/envs/torch_tf/lib/python3.7/site-packages/treetaggerwrapper.py:740: FutureWarning: Possible nested set at position 8
  re.IGNORECASE | re.VERBOSE)
/home/aboumada/anaconda3/envs/torch_tf/lib/python3.7/site-packages/treetaggerwrapper.py:2044: FutureWarning: Possible nested set at position 152
  re.VERBOSE | re.IGNORECASE)
/home/aboumada/anaconda3/envs/torch_tf/lib/python3.7/site-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/home/aboumada/anaconda3/envs/torch_tf/lib/python3.7/site-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [2]:
import pandas as pd
df = pd.read_csv("/data/aboumada/Data/3_Datasets/full_df_noFeatures2.csv")
#df_en = pd.read_csv('/data/aboumada/Data/3_Datasets/anonym_final_en_1.csv')
len(df)

12826

In [3]:
from easy_nlp.preprocessing import TextPreprocessing
tp=TextPreprocessing(df,'TEXT')
tp.fit_transform()

## Run if Random Sampling 

In [4]:
from sklearn.model_selection import train_test_split
df_train , df_test = train_test_split(df,random_state=1, test_size=0.2)


## Run if Out of event 

In [5]:
#events_test= ['Bruno', 'Eleanor']
#df_test = df[df.event.isin(events_test)]
#df_train = df[~df.event.isin(events_test)]

## Run if Out of Type 

In [6]:
#events_test= ['EffondrementMarseille']
#df_train=df_en[['text_clean','CAT']]

In [7]:
def get_sentences_labels(df,text_column='text_clean',label_column='CAT'):    
    dic_labels_to_cat = {value:x for x,value in enumerate(df[label_column].unique())}
    dic_cat_labels = {x:value for x,value in enumerate(df[label_column].unique())}
    df['text_clean']= df['text_clean'].map(lambda text_clean : re.sub('["#$%&()*+,-./:;<=>@[\]^_`{|}~\n\t’\']', '', text_clean))
    df2 = df[label_column].map(dic_labels_to_cat)
    sentences = df[text_column].values
    labels = df2.values.astype(int)
    return sentences,labels,dic_cat_labels

def custom_sentences_labels(df,dic_cat,dic_cat3,dic_cat2,text_column='text_clean',label_column='CAT'):
    dic_cat_labe = dic_cat
    dic_cat_labe2 = dic_cat2
    dic_cat_labe3 = dic_cat3
    #df['text_clean']= df['text_clean'].map(lambda text_clean : re.sub('["#$%&()*+,-./:;<=>@[\]^_`{|}~\n\t’\']', '', text_clean))
    dic_labels_to_cat = {v: k for k, v in dic_cat_labe.items()}
    dic_labels_to_cat2 = {v: k for k, v in dic_cat_labe2.items()}
    dic_labels_to_cat3 = {v: k for k, v in dic_cat_labe3.items()}
    sentences = df[text_column].values
    df_cat = df['CAT'].map(dic_labels_to_cat)
    labels_CAT = df_cat.values.astype(int)
    df_cat2 = df['CAT2'].map(dic_labels_to_cat2)
    labels_CAT2 = df_cat2.values.astype(int)
    df_cat3 = df['CAT3'].map(dic_labels_to_cat3)
    labels_CAT3 = df_cat3.values.astype(int)

    return sentences,(labels_CAT,labels_CAT3,labels_CAT2),dic_cat_labe

dic_cat_labels_CAT = {0: 'Soutiens', 1: 'Message-NonUtilisable', 2: 'Degats-Humains', 3: 'Degats-Materiels', 4: 'Avertissement-conseil', 5: 'AutresMessages',6: 'Critiques'}
dic_cat_labels_CAT3 = {0: 'Message-InfoUrgent', 1: 'Message-NonUtilisable', 2: 'Message-InfoNonUrgent'}
dic_cat_labels_CAT2 = {0: 'Message-Utilisable', 1: 'Message-NonUtilisable'}

#sentences_train,labels_train,dic_cat_labels=get_sentences_labels(df_train,text_column='text_clean',label_column='CAT')
sentences_train,labels_train,_=custom_sentences_labels(df_train,dic_cat_labels_CAT,dic_cat_labels_CAT3,dic_cat_labels_CAT2,text_column='processed_text',label_column='CAT')
sentences_test,test_labels,_=custom_sentences_labels(df_test,dic_cat_labels_CAT,dic_cat_labels_CAT3,dic_cat_labels_CAT2,text_column='processed_text',label_column='CAT')

In [8]:
from sklearn.model_selection import train_test_split
from easy_nlp.feature_extraction import BertInput

bert_input= BertInput(AutoTokenizer.from_pretrained('bert-base-multilingual-cased'))


X_train = bert_input.fit_transform(sentences_train)
X_test = bert_input.fit_transform(sentences_test)


In [9]:
# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels_cat, validation_labels_cat,train_labels_cat3, validation_labels_cat3,train_labels_cat2, validation_labels_cat2 = train_test_split(X_train[0], labels_train[0], labels_train[1], labels_train[2],
                                                            random_state=1, test_size=0.2)
# Do the same for the masks.
train_masks, validation_masks= train_test_split(X_train[1],
                                             random_state=1, test_size=0.2)

test_inputs = X_test[0]
test_masks = X_test[1]
test_labels_cat = test_labels[0]
test_labels_cat3 = test_labels[1]
test_labels_cat2 = test_labels[2]


In [10]:
import torch 
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
test_inputs = torch.tensor(test_inputs)

train_labels_cat = torch.tensor(train_labels_cat)
validation_labels_cat = torch.tensor(validation_labels_cat)
test_labels_cat = torch.tensor(test_labels_cat)

train_labels_cat3 = torch.tensor(train_labels_cat3)
validation_labels_cat3 = torch.tensor(validation_labels_cat3)
test_labels_cat3 = torch.tensor(test_labels_cat3)

train_labels_cat2 = torch.tensor(train_labels_cat2)
validation_labels_cat2 = torch.tensor(validation_labels_cat2)
test_labels_cat2 = torch.tensor(test_labels_cat2)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
test_masks = torch.tensor(test_masks)



In [11]:
def get_label_callback(dataset,idx):
    return dataset[idx][3].item()

In [12]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 16

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs,train_masks,train_labels_cat,train_labels_cat3,train_labels_cat2)
train_sampler = RandomSampler(train_data)
#train_sampler = ImbalancedDatasetSampler(train_data,callback_get_label=get_label_callback)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs,validation_masks ,validation_labels_cat,validation_labels_cat3,validation_labels_cat2)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


# Create the DataLoader for our test set.
test_data = TensorDataset(test_inputs,test_masks, test_labels_cat,test_labels_cat3,test_labels_cat2)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [13]:
model = ThreeTaskLearning(model=AutoModel.from_pretrained('bert-base-multilingual-cased'),
        dropout_rate = 0.1, 
        device = torch.device("cuda"),
        )
model.cuda()

ThreeTaskLearning(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [14]:
from transformers import AdamW,get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 3e-8 # args.adam_epsilon  - default is 1e-8.
                )


# Number of training epochs (authors recommend between 2 and 4)
epochs = 1

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs 

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [15]:
import numpy as np
train = Train()
train.fit_ThreeTask(model,train_dataloader,validation_dataloader,epochs,torch.device('cuda'),optimizer,scheduler)
print("")
print("Training complete!")


======== Epoch 1 / 1 ========
Training...
  Batch    40  of    513.    Elapsed: 0:00:10.
  Batch    80  of    513.    Elapsed: 0:00:20.
  Batch   120  of    513.    Elapsed: 0:00:29.
  Batch   160  of    513.    Elapsed: 0:00:39.
  Batch   200  of    513.    Elapsed: 0:00:49.
  Batch   240  of    513.    Elapsed: 0:00:58.
  Batch   280  of    513.    Elapsed: 0:01:07.
  Batch   320  of    513.    Elapsed: 0:01:16.
  Batch   360  of    513.    Elapsed: 0:01:26.
  Batch   400  of    513.    Elapsed: 0:01:35.
  Batch   440  of    513.    Elapsed: 0:01:44.
  Batch   480  of    513.    Elapsed: 0:01:53.

  Average training loss: 1.65
  Training epcoh took: 0:02:01

Running Validation...


/home/aboumada/anaconda3/envs/torch_tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aboumada/anaconda3/envs/torch_tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
save currently the best model to [tmp]
save model parameters to [tmp]


  Accuracy: 0.85
  F1: 0.80
  Recall: 0.82
  Precision: 0.81
  Validation took: 0:00:10

Training complete!


In [16]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(test_dataloader)))

# Put model in evaluation mode
model.eval()


# Tracking variables 
predictions_cat,predictions_cat3,predictions_cat2 , true_labels_cat,true_labels_cat3,true_labels_cat2  = [], [],[],[],[],[]

# Predict 
for batch in test_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(torch.device("cuda")) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask,b_labels_cat,b_labels_cat3,b_labels_cat2 = batch

    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model((b_input_ids,b_input_mask))
        logits_cat = outputs[0]
        logits_cat3 = outputs[1]
        logits_cat2 = outputs[2]

    # Move logits and labels to CPU
    logits_cat = logits_cat.detach().cpu().numpy()
    label_ids_cat = b_labels_cat.to('cpu').numpy()
    predictions_cat.extend(logits_cat)
    true_labels_cat.extend(label_ids_cat)
    
    logits_cat3 = logits_cat3.detach().cpu().numpy()
    label_ids_cat3 = b_labels_cat3.to('cpu').numpy()
    predictions_cat3.extend(logits_cat3)
    true_labels_cat3.extend(label_ids_cat3)
    
    
    logits_cat2 = logits_cat2.detach().cpu().numpy()
    label_ids_cat2 = b_labels_cat2.to('cpu').numpy()
    predictions_cat2.extend(logits_cat2)
    true_labels_cat2.extend(label_ids_cat2)


print('    DONE.')



Predicting labels for 161 test sentences...
    DONE.


In [17]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

pred_flat_cat = np.argmax(predictions_cat, axis=1)
true_labels_cat=[dic_cat_labels_CAT.get(x) for x in true_labels_cat]
pred_flat_cat = [dic_cat_labels_CAT.get(x) for x in pred_flat_cat]


cr= classification_report(true_labels_cat,pred_flat_cat,digits=4)
print(accuracy_score(pred_flat_cat,true_labels_cat))
print(cr)

0.8191738113795791
                       precision    recall  f1-score   support

       AutresMessages     0.5000    0.2079    0.2937       178
Avertissement-conseil     0.7326    0.7104    0.7214       297
            Critiques     0.0000    0.0000    0.0000        28
       Degats-Humains     0.7027    0.5306    0.6047        49
     Degats-Materiels     0.6923    0.4839    0.5696        93
Message-NonUtilisable     0.8509    0.9464    0.8961      1827
             Soutiens     0.7714    0.5745    0.6585        94

             accuracy                         0.8192      2566
            macro avg     0.6071    0.4934    0.5348      2566
         weighted avg     0.7921    0.8192    0.7982      2566



/home/aboumada/anaconda3/envs/torch_tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

pred_flat_cat3 = np.argmax(predictions_cat3, axis=1)
true_labels_cat3 =[dic_cat_labels_CAT3.get(x) for x in true_labels_cat3]
pred_flat_cat3 = [dic_cat_labels_CAT3.get(x) for x in pred_flat_cat3]


cr= classification_report(true_labels_cat3,pred_flat_cat3,digits=4)
print(accuracy_score(pred_flat_cat3,true_labels_cat3))
print(cr)

0.8312548713951676
                       precision    recall  f1-score   support

Message-InfoNonUrgent     0.6528    0.4200    0.5112       300
   Message-InfoUrgent     0.7349    0.7198    0.7273       439
Message-NonUtilisable     0.8703    0.9256    0.8971      1827

             accuracy                         0.8313      2566
            macro avg     0.7527    0.6885    0.7118      2566
         weighted avg     0.8217    0.8313    0.8229      2566



In [19]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

pred_flat_cat2 = np.argmax(predictions_cat2, axis=1)
true_labels_cat2 =[dic_cat_labels_CAT2.get(x) for x in true_labels_cat2]
pred_flat_cat2 = [dic_cat_labels_CAT2.get(x) for x in pred_flat_cat2]


cr= classification_report(true_labels_cat2,pred_flat_cat2,digits=4)
print(accuracy_score(pred_flat_cat2,true_labels_cat2))
print(cr)

0.8495713172252534
                       precision    recall  f1-score   support

Message-NonUtilisable     0.8863    0.9048    0.8954      1827
   Message-Utilisable     0.7518    0.7131    0.7319       739

             accuracy                         0.8496      2566
            macro avg     0.8191    0.8089    0.8137      2566
         weighted avg     0.8476    0.8496    0.8484      2566



In [20]:
model.save('model.pth')

save model parameters to [model.pth]
